In [24]:
import numpy as np
import xarray as xr
import postmit as pm
import pickle
import xgcm
from MITgcmutils import jmd95 as jmd
import xrft

In [3]:
path = "/path/to/model/output/"
path_obs = "/path/to/obs/"

### Figure 1

We need a 5d mean of `THETA` and `SIarea` as well as the mean residual overturning circulation remapped to z-levels.

In [4]:
ds_1y = pm.checks.apply_all_checks(xr.open_zarr(path + "zarr_Diags/output.1y.zarr/"), 
                                   path_to_input=path + "/input/")
ds_5d = pm.checks.apply_all_checks(xr.open_zarr(path + "zarr_Diags/output.5d.zarr/"), 
                                   path_to_input=path + "/input/")

Defining a xgcm-grid for interpolation etc.

In [5]:
metrics = {
    ('X'): ['dxC', 'dxG', 'dxF', 'dxV'], # X distances
    ('Y'): ['dyC', 'dyG', 'dyF', 'dyU'], # Y distances
    ('Z'): ['drF', 'drW', 'drS', 'drC'], # Z distances
    ('X', 'Y'): ['rAw', 'rAs', 'rA', 'rAz'] # Areas in x-y plane
}

grid = xgcm.Grid(ds_5d, periodic=["X"], metrics=metrics)

We compute the residual overturning based on $\sigma_{2}$.

In [6]:
ds_5d = pm.calcs.sigi(ds_5d, 2)
ds_5d["SIG2"] = ds_5d.SIG2.where(ds_5d.SIG2 > 1010.)

These are the bounds of the isopycnal layers that we will project onto. Note that we obviously do not expect $\sigma_{2}$ to be 0 or 100 but we include the extremes to make sure we capture the full circulation.

In [7]:
layer_bounds = np.hstack(([0., 10., 20., 25., 30., 31.], 
                           np.arange(32., 35., 0.1),
                           np.arange(35., 36., 0.05),
                           np.arange(36., 36.9, 0.02),
                           np.arange(36.9, 37.199, 0.01),
                           np.arange(37.2, 37.6, 0.02),
                           np.arange(37.6, 37.8, 0.01),
                           np.arange(37.8, 37.899, 0.05),
                           [37.9, 38., 38.1, 38.3, 38.5, 39., 40., 45., 50., 100.]))

We create a mask to exclude the shelf. The dataset does already include masks but due to the interpolation, we need to extend the existing mask one grid point "into the ocean".

In [8]:
maskShelf = np.ones(np.shape(ds_5d.maskC))
for k in np.arange(0, np.shape(maskShelf)[0]):
    firstwet = np.argmax(ds_5d.VVEL[0, k, :, 0].values != 0.)
    maskShelf[k, 0:firstwet+1, :] = 0

ds_5d["maskShelf"] = xr.DataArray(maskShelf, dims=("Z", "YG", "XC"))

In [ ]:
dz = ds_5d.drF
ds_5d["VTRANS"] = (ds_5d.VVEL * dz).where(ds_5d.maskShelf != 0).rename("VTRANS")
ds_5d["levThick"] = (ds_5d.drF * ds_5d.hFacS * ds_5d.maskS).expand_dims(dim={"time": ds_5d.time})

part1 = grid.interp(ds_5d.SIG2, "Z", to="left", boundary="extend").rename({"Zl": "Zp1"})
part2 = grid.interp(ds_5d.SIG2, "Z", to="right", boundary="extend").isel(Zu=slice(-1, None)).rename({"Zu": "Zp1"})
SIG2b = xr.concat((part1, part2), dim="Zp1")
SIG2vb = grid.interp(SIG2b - 1000., "Y", boundary="extend").rename("SIG2").chunk({"Zp1": -1})

VTRANS_rho = grid.transform(ds_5d.VTRANS, "Z", layer_bounds, target_data=SIG2vb, method='conservative').rename({"SIG2": "layer_center"})
iso = xr.Dataset(coords={"layer_bounds": layer_bounds, "layer_center": VTRANS_rho.layer_center})
iso["VTRANS"] = VTRANS_rho.chunk({"time": 720, "YG": 320, "XC":240, "layer_center": 1})
iso["levThick"] = grid.transform(ds_5d.levThick, "Z", layer_bounds, target_data=SIG2vb, method='conservative').rename({"SIG2": "layer_center"})

iso["MOC_res"] = -grid.integrate(iso.VTRANS.cumsum(dim="layer_center"), "X").chunk({"layer_center": 182})
iso["MOC_res"] = iso.MOC_res.where(iso.VTRANS.sum("XC")!=0)

iso["layer_depths"] = xr.DataArray(-iso.levThick.mean("time").sortby("layer_center", ascending=True).cumsum(dim="layer_center"))
iso["levThickmean"] = xr.DataArray(iso.levThick.mean("time")))
metrics_tmp = {
    ('Z'): ['levThickmean'], # Z distances
    }
iso["layer_center"].attrs["axis"] = "Z"
iso["layer_bounds"].attrs["axis"] = "Z"
iso["layer_bounds"].attrs["c_grid_axis_shift"] = -0.5
gridtmp = xgcm.Grid(iso, periodic=False, metrics=metrics_tmp)
MOC_res_z = gridtmp.transform(iso["MOC_res"], "Z", ds_5d.Z, target_data=iso.layer_depths.mean("XC"))
ds_mean_MOC_res_z = MOC_res_z.mean("time")
ds_mean_MOC_res_z.to_netcdf("MOC_res_z_0201-01-01_0300-12-30.nc")
MOC_res_z_JJA = MOC_res_z.groupby("time.season").mean("time").sel(season="JJA")
MOC_res_JJA = iso.MOC_res.groupby("time.season").mean("time").sel(season="JJA")
MOC_res_z_JJA.to_netcdf("MOC_res_z_0201-01-01_0300-12-30_JJA.nc")
MOC_res_JJA.to_netcdf("MOC_res_0201-01-01_0300-12-30_JJA.nc")

We will take `0250-09-03` as the snapshot and compute the mean over the whole 100 years.

In [55]:
t1_mean = '0201-01-01'
t2_mean = '0300-12-30'
t_snap = '0250-09-03' 

In [56]:
ds_snap = ds_5d.sel(time=t_snap).mean("time")
ds_mean = ds_1y.sel(time=slice(t1_mean, t2_mean)).mean("time")

In [28]:
x2 = 2.4e6
y1 = 0.01e6

In [ ]:
THETAyz = ds_snap.THETA.sel(XC=x2, method='nearest')
THETAxz = ds_snap.THETA.sel(YC=y1, method='nearest')
THETAsurf = ds_snap.THETA.isel(Z=0)

Now we save the computed variables to plot them later.

In [ ]:
THETAsurf.to_netcdf("THETA_0250-09-03_surf.nc")
THETAyz.to_netcdf("THETA_0250-09-03_yz.nc")
THETAxz.to_netcdf("THETA_0250-09-03_xz.nc")
ds_snap.SIarea.to_netcdf("SIarea_0250-09-03.nc")

### Figure 2
For figure 2 we need the model average, zonal mean `THETA`, `SALT` and `UVEL` as well as the winter (JJA) mean, zonal average `SIarea` and `SIheff`. We also use observational data: temperature and salinity from WOA18, zonal velocity from SLTAC and sea ice concentration from NSIDC.

In [57]:
ds_meanX = ds_mean.reset_coords().mean(("XC", "XG"))
for coord in ds_1y.coords:
    if coord in ds_meanX.variables:
        ds_meanX = ds_meanX.set_coords(coord)

In [ ]:
ds_meanX.THETA.to_netcdf("THETA_0201-01-01_0300-12-30_zonal_mean.nc")
ds_meanX.SALT.to_netcdf("SALT_0201-01-01_0300-12-30_zonal_mean.nc")
ds_meanX.UVEL.to_netcdf("UVEL_0201-01-01_0300-12-30_zonal_mean.nc")

In [30]:
ds_JJA = ds_5d.groupby("time.season").mean("time").sel(season="JJA").reset_coords().mean(("XC", "XG"))
for coord in ds_5d.coords:
    if coord in ds_JJA.variables:
        ds_JJA = ds_JJA.set_coords(coord)

In [ ]:
ds_JJA.SIarea.to_netcdf("SIarea_0201-01-01_0300-12-30_JJA_zonal_mean.nc")
ds_JJA.SIheff.to_netcdf("SIheff_0201-01-01_0300-12-30_JJA_zonal_mean.nc")

#### WOA18  
We use the 1985-1994 average from WOA18 as that corresponds to the initial conditions. This average was used because the atmospheric forcing is the repeated year 1991. Also, we only use the WOA18 data up to the maximum depth of the simulation.

In [ ]:
lat_min = -75
lat_max = -35
lon_cen = -30
z_max = -(ds_1y.Z.min().values)

woa = xr.open_mfdataset(path_obs + "WOA18/woa18_8594_[ts]1[3-6]_01.nc", 
                        decode_times=False, combine='by_coords').squeeze()
woa_s = woa.sel(lat=slice(lat_min, lat_max)).sel(lon=lon_cen, method='nearest')

In [ ]:
woa_s.mean("time").sel(depth=slice(0, z_max)).t_an.to_netcdf("WOA18_t_an.nc")
woa_s.mean("time").sel(depth=slice(0, z_max)).s_an.to_netcdf("WOA18_s_an.nc")

#### SLTAC
The zonal velocities are averaged across 60 degrees West to 0 degrees West to compare better to the symmetric re-entrant channel configuration. Just a slice at 30 degrees West does not compare too well to the simulation due to local circulation features related to topography (which is not present in the simulation).

In [ ]:
sltac = xr.open_mfdataset(path_obs + "SLTAC/" 
                          + "cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D*_mean_uv.nc")
u_mean_sltac = sltac.ugos.mean("time").sel(latitude=slice(-75, -35), longitude=slice(-60, 0)).mean("longitude")

In [ ]:
u_mean_sltac.to_netcdf("SLTAC_ugos.nc")

#### NSIDC
We select the same period as for WOA18 (1985-1994)

In [ ]:
siconc = xr.open_mfdataset(path_obs + "NSIDC/seaice_conc_monthly_sh_197811_202205_v04r00.nc")

In [ ]:
SI_JJA = siconc.isel(tdim=slice((6 * 12) + 2, (16 * 12) + 2)).groupby("time.season").mean("tdim").sel(season="JJA")
conc_JJA = SI_JJA.cdr_seaice_conc_monthly.where(SI_JJA.cdr_seaice_conc_monthly<=1.0, other=np.nan)
conc_JJA_030W = conc_JJA.where((SI_JJA.longitude<=-29) & (SI_JJA.longitude>=-31)).mean("x")

The NSIDC data is on a polar stereographic grid, so we inteprolate it to a latitude that is equivalent to the forcing used (75 - 35 deg. S) to make it easier to plot later on.

In [ ]:
interp_lat = np.linspace(lat_min, lat_max, 320)
SI_JJA = siconc.groupby("time.season").mean("tdim").sel(season="JJA")
conc_JJA = SI_JJA.cdr_seaice_conc_monthly.where(SI_JJA.cdr_seaice_conc_monthly<=1.0, other=np.nan)
conc_JJA_030W = conc_JJA.where((SI_JJA.longitude<=-29) & (SI_JJA.longitude>=-31)).mean("x")
lat_030W = siconc.latitude.where((SI_JJA.longitude<=-29) & (SI_JJA.longitude>=-31)).mean("x")

In [ ]:
conc_interp = np.interp(interp_lat, lat_030W.values[::-1], conc_JJA_030W.values[::-1])

In [ ]:
NSICD_JJA = xr.DataArray(conc_interp, dims=("latitude"), coords={"latitude": interp_lat})

In [ ]:
NSICD_JJA.rename("siconc").to_netcdf("NSIDC_siconc_interp_lat.nc")

#### Zonal wind from the forcing (based on ERA5)

In [ ]:
forcing = xr.open_dataset(path + "input/ESII_SO-RCSI100.L55_ctlv30_030W-era5-atmospheric_forcing.nc")
zonal_wind = forcing.zonal_wind.mean("time").isel(x=0)

In [ ]:
zonal_wind.to_netcdf("ERA5_zonal_wind.nc")

### Figure 3
For figure 3 we need to compute some statistics from all the eddies that were detected.

In [4]:
with open(path
          + 'tracking/tracks_02010101_03001230_all.pickle', 'rb') as f:
    tracks = pickle.load(f)
f.close()

For the statistics, we use the mean locations, amplitudes and diameters from each track. And we also extract the mean sea ice concentration above each eddy to differentiate between eddies in the open ocean and under sea ice.

In [ ]:
number = len(tracks)
all_tracks = xr.Dataset({"number": (["number"], np.arange(0, number)),
                         "type": (["number"], np.stack([tracks[i]["type"] 
                                                        for i in np.arange(number)])),
                         "time": (["number"], np.stack([tracks[i]["time"][0] 
                                                        + ((tracks[i]["time"][-1] - tracks[i]["time"][0]) / 2) 
                                                        for i in np.arange(number)])),
                         "lifetime": (["number"], np.stack([(tracks[i]["time"][-1] 
                                                             - tracks[i]["time"][0]).days 
                                                            for i in np.arange(number)])),
                         "lon": (["number"], np.hstack([np.mean(tracks[i]["lon"]) 
                                                        for i in np.arange(number)])),
                         "lat": (["number"], np.hstack([np.mean(tracks[i]["lat"]) 
                                                        for i in np.arange(number)])),
                         "i": (["number"], np.hstack([int(np.mean([np.median(tracks[i]["eddy_i"][j]) 
                                                      for j in np.arange(len(tracks[i]["eddy_i"]))]))  
                                                      for i in np.arange(number)])),
                         "j": (["number"], np.hstack([int(np.mean([np.median(tracks[i]["eddy_j"][j]) 
                                                      for j in np.arange(len(tracks[i]["eddy_j"]))]))  
                                                      for i in np.arange(number)])),
                         "scale": (["number"], np.hstack([np.mean(tracks[i]["scale"]) 
                                                          for i in np.arange(number)])),
                         "amp": (["number"], np.hstack([np.mean(tracks[i]["amp"]) 
                                                        for i in np.arange(number)]))
                        })

all_tracks["SIarea"] = ds_5d.SIarea.isel(XC=all_tracks.i, 
                                         YC=all_tracks.j).sel(time=all_tracks.time, 
                                                              method="nearest").drop(["XC", "YC", "time"])

all_tracks.to_netcdf("all_tracks_02010101_03001230.nc")

Now we compute the distribution of diameter and amplitude for the tracks, separating by type (anticyclonic vs. cyclonic) and sea ice cover (open ocean vs. under ice)

In [ ]:
cycl = all_tracks.where(all_tracks.type == 'cyclonic')
anti = all_tracks.where(all_tracks.type == 'anticyclonic')
cycl_oo = cycl.where(cycl.SIarea < 0.15).dropna("number")
anti_oo = anti.where(anti.SIarea < 0.15).dropna("number")
cycl_ice = cycl.where(cycl.SIarea > 0.8).dropna("number")
anti_ice = anti.where(anti.SIarea > 0.8).dropna("number")

In [ ]:
scale_bins = np.linspace(0, 150, 51)
scale_labels = (scale_bins[0:-1] + scale_bins[1::]) / 2
scales = xr.Dataset(coords={"scale": scale_labels})

scales["scale_cyc_oo"] = cycl_oo.groupby_bins("scale", bins=scale_bins, 
                                              labels=scale_labels).count().scale.rename({"scale_bins": "scale"})
scales["scale_anti_oo"] = anti_oo.groupby_bins("scale", bins=scale_bins, 
                                               labels=scale_labels).count().scale.rename({"scale_bins": "scale"})

scales["scale_cyc_ice"] = cycl_ice.groupby_bins("scale", bins=scale_bins, 
                                                labels=scale_labels).count().scale.rename({"scale_bins": "scale"})
scales["scale_anti_ice"] = anti_ice.groupby_bins("scale", bins=scale_bins, 
                                                 labels=scale_labels).count().scale.rename({"scale_bins": "scale"})

In [ ]:
scales.to_netcdf("eddy_scale_distribution.nc")

In [ ]:
amp_bins = np.linspace(0, 5.5e-5, 51)
amp_labels = (amp_bins[0:-1] + amp_bins[1::]) / 2
amps = xr.Dataset(coords={"amp": amp_labels})

amps["amp_cyc_oo"] = cycl_oo.groupby_bins("amp", bins=amp_bins, 
                                              labels=amp_labels).count().amp.rename({"amp_bins": "amp"})
amps["amp_anti_oo"] = anti_oo.groupby_bins("amp", bins=amp_bins, 
                                               labels=amp_labels).count().amp.rename({"amp_bins": "amp"})

amps["amp_cyc_ice"] = cycl_ice.groupby_bins("amp", bins=amp_bins, 
                                                labels=amp_labels).count().amp.rename({"amp_bins": "amp"})
amps["amp_anti_ice"] = anti_ice.groupby_bins("amp", bins=amp_bins, 
                                                 labels=amp_labels).count().amp.rename({"amp_bins": "amp"})

In [ ]:
amps.to_netcdf("eddy_amp_distribution.nc")

And we reference each track to (0, 0) to plot all tracks and average the tracks.

In [21]:
a_oo_tralo = np.zeros((int(np.ceil(len(tracks)/10)), 90)) + np.nan
a_oo_trala = np.zeros((int(np.ceil(len(tracks)/10)), 90)) + np.nan
c_oo_tralo = np.zeros((int(np.ceil(len(tracks)/10)), 90)) + np.nan
c_oo_trala = np.zeros((int(np.ceil(len(tracks)/10)), 90)) + np.nan
a_ice_tralo = np.zeros((int(np.ceil(len(tracks)/10)), 90)) + np.nan
a_ice_trala = np.zeros((int(np.ceil(len(tracks)/10)), 90)) + np.nan
c_ice_tralo = np.zeros((int(np.ceil(len(tracks)/10)), 90)) + np.nan
c_ice_trala = np.zeros((int(np.ceil(len(tracks)/10)), 90)) + np.nan
a_oo = 0
c_oo = 0
a_ice = 0
c_ice = 0
for tt in np.arange(0, len(tracks), 10):
    lonn_start = tracks[tt]['lon'][0]
    latt_start = tracks[tt]['lat'][0]
    tttt_start = tracks[tt]['time'][0]
    lonn_end = tracks[tt]['lon'][-1]
    latt_end = tracks[tt]['lat'][-1]
    tttt_end = tracks[tt]['time'][-1]
    ice_start = ds_5d.SIarea.sel(XC=lonn_start, 
                                 method='nearest').sel(YC=latt_start, 
                                                       method='nearest').sel(time=tttt_start).values
    ice_end = ds_5d.SIarea.sel(XC=lonn_end, 
                               method='nearest').sel(YC=latt_end, 
                                                     method='nearest').sel(time=tttt_end).values
    if ((ice_start < 0.15) & (ice_end < 0.15)):
        if ((tracks[tt]['type'] == 'anticyclonic') & (len(tracks[tt]['time']) > 1)):
            a_oo_tralo[int(tt/10), 0:len(tracks[tt]['time'])] = tracks[tt]['lon'] - tracks[tt]['lon'][0]
            a_oo_trala[int(tt/10), 0:len(tracks[tt]['time'])] = tracks[tt]['lat'] - tracks[tt]['lat'][0]
            a_oo += 1
        elif ((tracks[tt]['type'] == 'cyclonic') & (len(tracks[tt]['time']) > 1)):
            c_oo_tralo[int(tt/10), 0:len(tracks[tt]['time'])] = tracks[tt]['lon'] - tracks[tt]['lon'][0]
            c_oo_trala[int(tt/10), 0:len(tracks[tt]['time'])] = tracks[tt]['lat'] - tracks[tt]['lat'][0]
            c_oo += 1
    elif ((ice_start > 0.8) & (ice_end > 0.8)):
        if ((tracks[tt]['type'] == 'anticyclonic') & (len(tracks[tt]['time']) > 1)):
            a_ice_tralo[int(tt/10), 0:len(tracks[tt]['time'])] = tracks[tt]['lon'] - tracks[tt]['lon'][0]
            a_ice_trala[int(tt/10), 0:len(tracks[tt]['time'])] = tracks[tt]['lat'] - tracks[tt]['lat'][0]
            a_ice += 1
        elif ((tracks[tt]['type'] == 'cyclonic') & (len(tracks[tt]['time']) > 1)):
            c_ice_tralo[int(tt/10), 0:len(tracks[tt]['time'])] = tracks[tt]['lon'] - tracks[tt]['lon'][0]
            c_ice_trala[int(tt/10), 0:len(tracks[tt]['time'])] = tracks[tt]['lat'] - tracks[tt]['lat'][0]
            c_ice += 1

In [22]:
tracks_referenced = xr.Dataset(coords={"time": np.arange(0, 90), "track": np.arange(0, len(tracks), 10)})
tracks_referenced["a_oo_tralo"] = (["track", "time"], a_oo_tralo)
tracks_referenced["a_oo_trala"] = (["track", "time"], a_oo_trala)
tracks_referenced["c_oo_tralo"] = (["track", "time"], c_oo_tralo)
tracks_referenced["c_oo_trala"] = (["track", "time"], c_oo_trala)
tracks_referenced["a_ice_tralo"] = (["track", "time"], a_ice_tralo)
tracks_referenced["a_ice_trala"] = (["track", "time"], a_ice_trala)
tracks_referenced["c_ice_tralo"] = (["track", "time"], c_ice_tralo)
tracks_referenced["c_ice_trala"] = (["track", "time"], c_ice_trala)
tracks_referenced["a_oo_number"] = a_oo
tracks_referenced["c_oo_number"] = c_oo
tracks_referenced["a_ice_number"] = a_ice
tracks_referenced["c_ice_number"] = c_ice

In [23]:
tracks_referenced.to_netcdf("tracks_referenced_to_00_every10th.nc")

### Figure 4
Here we need the zonally integrated, depth-integrated average heat transports.

In [ ]:
Ms = xr.open_mfdataset(path + 'post/MHTs.0201-0300.nc')

We need dz for the vertical integration (`hFacS * drF`). (The zonal integration has already been done during the calculations.)

In [58]:
hfac = ds_meanX.hFacS.values
dz = ds_1y.drF.values

Zonal integral of net heat flux into the ocean

In [ ]:
oq = (ds_1y.sel(time=slice(t1_mean, t2_mean)).oceQnet * ds_1y.dxF).sum("XC")

We multiply everything by `1e-12` to convert values to TW.

In [ ]:
mean_HT = xr.Dataset(coords={"YC": ds_1y.YC, "YG": ds_1y.YG})
mean_HT["MHT"] = ((Ms.MHT * dz[:, None] * hfac).sum("Z") * 1e-12).mean("time")
mean_HT["MHTstd"] = ((Ms.MHT * dz[:, None] * hfac).sum("Z") * 1e-12).std("time")
mean_HT["MHTek"] = (Ms.MHTek * 1e-12).mean("time")
mean_HT["THT"] = ((Ms.THT * dz[:, None] * hfac).sum("Z") * 1e-12).mean("time")
mean_HT["THTstd"] = ((Ms.THT * dz[:, None] * hfac).sum("Z") * 1e-12).std("time")
mean_HT["totalHT"] = (((Ms.MHT + Ms.THT) * dz[:, None] * hfac).sum("Z") * 1e-12).mean("time")
mean_HT["totalHTstd"] = (((Ms.MHT + Ms.THT) * dz[:, None] * hfac).sum("Z") * 1e-12).std("time")
mean_HT["OQ"] = (oq.mean("time") * ds_1y.dyF.mean("XC")).cumsum("YC") * 1e-12

In [ ]:
mean_HT.to_netcdf("mean_heat_transports_0201-01-01_0300-12-30_zonal_vertical_integral.nc")

### Figure 5

Same as for Figure 4 but with the different components of transient HT.

In [ ]:
Ts = xr.open_mfdataset(path + 'post/THTs.0201-0300.nc').drop("z")

In [ ]:
HTtrap = xr.open_mfdataset(path + "/post/THTobs.0201-0300.nc", 
                           concat_dim="time", combine="nested", data_vars='minimal', 
                           coords='minimal', compat='override').rename({"lat": "YG", "z": "Z"})

In [ ]:
transient_HT = xr.Dataset(coords={"YG": ds_1y.YG})
transient_HT["THT"] = ((Ts.THT * dz[:, None] * hfac).sum("Z") * 1e-12).mean("time")
transient_HT["THTdiv"] = ((Ts.THTdiv * dz[:, None] * hfac).sum("Z") * 1e-12).mean("time")
transient_HT["THTdivstd"] = ((Ts.THTdiv * dz[:, None] * hfac).sum("Z") * 1e-12).std("time")
transient_HT["THTref"] = ((Ts.THTref * dz[None, None, :] * hfac.T[None, :, :]).sum("Z") * 1e-12).mean("time")
transient_HT["THTrefstd"] = ((Ts.THTref * dz[None, None, :] * hfac.T[None, :, :]).sum("Z") * 1e-12).std("time")
transient_HT["THTeddy"] = ((Ts.THTeddy * dz[:, None] * hfac).sum("Z") * 1e-12).mean("time")
transient_HT["THTeddystd"] = ((Ts.THTeddy * dz[:, None] * hfac).sum("Z") * 1e-12).std("time")
transient_HT["THTdiveddy"] = ((Ts.THTdiveddy * dz[:, None] * hfac).sum("Z") * 1e-12).mean("time")
transient_HT["THTdiveddystd"] = ((Ts.THTdiveddy * dz[:, None] * hfac).sum("Z") * 1e-12).std("time")
transient_HT["THTrefeddy"] = ((Ts.THTrefeddy * dz[None, None, :] * hfac.T[None, :, :]).sum("Z") * 1e-12).mean("time")
transient_HT["THTrefeddystd"] = ((Ts.THTrefeddy * dz[None, None, :] * hfac.T[None, :, :]).sum("Z") * 1e-12).std("time")
transient_HT["THTtrack"] = ((Ts.THTtrack * dz[:, None] * hfac).sum("Z") * 1e-12).mean("time")
transient_HT["THTtrackstd"] = ((Ts.THTtrack * dz[:, None] * hfac).sum("Z") * 1e-12).std("time")
transient_HT["THTdivtrack"] = ((Ts.THTdivtrack * dz[:, None] * hfac).sum("Z") * 1e-12).mean("time")
transient_HT["THTdivtrackstd"] = ((Ts.THTdivtrack * dz[:, None] * hfac).sum("Z") * 1e-12).std("time")
transient_HT["THTtrap"] = ((HTtrap["THTobs"] * dz * hfac.T).sum("Z") * 1e-12).mean("time")
transient_HT["THTtrapstd"] = ((HTtrap["THTobs"] * dz * hfac.T).sum("Z") * 1e-12).std("time")

In [ ]:
transient_HT.to_netcdf("transient_heat_transports_0201-01-01_0300-12-30_zonal_vertical_integral.nc")

### Figure 6
Again, similar to the preparation for Figures 4 and 5 but for winter (JJA) values below the mixed layer and also isopycnal HT.

In [ ]:
Ts_NoML = xr.open_mfdataset(path + 'post/THTsNoMLJJA.0201-0300.nc').drop("z")
iso_NoML = xr.open_mfdataset(path + "post/HTisoNoMLjja_eddy.0201-0300.nc").drop("z")

In [ ]:
transient_HT_NoML = xr.Dataset(coords={"YG": ds_1y.YG})
transient_HT_NoML["THT"] = ((Ts_NoML.THT * dz[:, None] * hfac).sum("Z") * 1e-12).mean("time")
transient_HT_NoML["THTdiv"] = ((Ts_NoML.THTdiv * dz[:, None] * hfac).sum("Z") * 1e-12).mean("time")
transient_HT_NoML["THTdivstd"] = ((Ts_NoML.THTdiv * dz[:, None] * hfac).sum("Z") * 1e-12).std("time")
transient_HT_NoML["THTeddy"] = ((Ts_NoML.THTeddy * dz[:, None] * hfac).sum("Z") * 1e-12).mean("time")
transient_HT_NoML["THTeddystd"] = ((Ts_NoML.THTeddy * dz[:, None] * hfac).sum("Z") * 1e-12).std("time")
transient_HT_NoML["THTdiveddy"] = ((Ts_NoML.THTdiveddy * dz[:, None] * hfac).sum("Z") * 1e-12).mean("time")
transient_HT_NoML["THTdiveddystd"] = ((Ts_NoML.THTdiveddy * dz[:, None] * hfac).sum("Z") * 1e-12).std("time")

In [ ]:
isopycnal_HT_NoML = xr.Dataset(coords={"YG": ds_1y.YG})
isopycnal_HT_NoML["iso_total"] = ((iso_NoML.ADViso + iso_NoML.DIFFiso).sum("layer_center") * 1e-12).mean("time")
isopycnal_HT_NoML["iso_totalstd"] = ((iso_NoML.ADViso + iso_NoML.DIFFiso).sum("layer_center") * 1e-12).std("time")
isopycnal_HT_NoML["iso_totaleddy"] = ((iso_NoML.ADViso_eddyV 
                                      + iso_NoML.DIFFiso_eddy_opt4).sum("layer_center") * 1e-12).mean("time")
isopycnal_HT_NoML["iso_totaleddystd"] = ((iso_NoML.ADViso_eddyV 
                                         + iso_NoML.DIFFiso_eddy_opt4).sum("layer_center") * 1e-12).std("time")

In [ ]:
transient_HT_NoML.to_netcdf("transient_heat_transports_0201-01-01_0300-12-30_NoML_zonal_vertical_integral.nc")
isopycnal_HT_NoML.to_netcdf("isopycnal_heat_transports_0201-01-01_0300-12-30_NoML_zonal_vertical_integral.nc")

### Figure 7

We need mean winter mixed layer depth `MXLDEPTH` and temperature `THETA` as well as $\sigma_{2}$ on the v-grid

In [ ]:
metricsJJA = {
    ('Y'): ['dyC', 'dyG', 'dyF', 'dyU'], # Y distances
    ('Z'): ['drF', 'drW', 'drS', 'drC'], # Z distances
    }
gridJJA = xgcm.Grid(ds_JJA, periodic=False, metrics=metricsJJA)

In [ ]:
MLDonV = gridJJA.interp(ds_JJA.MXLDEPTH.where(ds_JJA.MXLDEPTH!=0), "Y", boundary="extend")
THETAonV = gridJJA.interp(ds_JJA.THETA.where(ds_JJA.THETA!=0), "Y", boundary="extend")

In [ ]:
MLDonV.where(MLDonV, other=0).rename("MXLDEPTH").to_netcdf("MXLDEPTH_0201-01-01_0300-12-30_JJA_zonal_mean.nc")
THETAonV.where(THETAonV, other=0).rename("THETA").to_netcdf("THETA_0201-01-01_0300-12-30_JJA_zonal_mean.nc")

In [ ]:
ds_JJA["SIG2"] = ds_5d.SIG2.groupby("time.season").mean("time").sel(season="JJA").mean(("XC"))
ds_JJA["SIG2"] = ds_JJA.SIG2.where(ds_JJA.SIG2 > 1010.)
SIG2onV = gridJJA.interp(ds_JJA.SIG2.where(ds_JJA.SIG2!=0), "Y", boundary="extend")
SIG2onV.where(SIG2onV, other=0).rename("SIG2").to_netcdf("SIG2_0201-01-01_0300-12-30_JJA_zonal_mean.nc")

as well as the mean THETA on isopycnals

In [13]:
target_rho_levelsC = np.hstack(([5., 15., 22.5, 27.5, 30.5, 31.5], 
                                np.arange(32.05, 35., 0.1),
                                np.arange(35.025, 36., 0.05),
                                np.arange(36.01, 36.9, 0.02),
                                np.arange(36.905, 37.2, 0.01),
                                np.arange(37.21, 37.6, 0.02),
                                np.arange(37.605, 37.8, 0.01),
                                np.arange(37.825, 37.9, 0.05),
                                [37.95, 38.05, 38.2, 38.4, 38.75, 39.5, 42.5, 47.5, 75.]))

In [ ]:
def z2rho(grid, var, target_levels, target_data, target_name, method):
    var_rho = grid.transform(var, "Z", target_levels, target_data=target_data, method=method)
    var_rho = var_rho.rename({target_name: "layer_center"})
    return var_rho

In [ ]:
metrics = {
       ('X'): ['dxC', 'dxG', 'dxF', 'dxV'], # X distances
       ('Y'): ['dyC', 'dyG', 'dyF', 'dyU'], # Y distances
       ('Z'): ['drF', 'drW', 'drS', 'drC'], # Z distances
       }
grid = xgcm.Grid(ds_5d, periodic=["X"], metrics=metrics)

In [ ]:
t_rho = z2rho(grid, ds_5d.THETA, target_rho_levelsC, ds_5d.SIG2 - 1000., 
              "SIG2", "linear").chunk({"time": 1, "XC": 240, "YC": 320})

In [ ]:
t_rhoonV = t_rho.groupby("time.season").mean("time").sel(season="JJA").mean(("XC"))
t_rhoonVJJA  = grid.interp(t_rhoonV.where(t_rhoonV!=0), "Y", boundary="extend").chunk({"YG": 320}).rename("t_rho")

In [ ]:
t_rhoonVJJA.to_netcdf("T_on_isopycnals_0201-01-01_0300-12-30_JJA_zonal_mean.nc")

We also need the winter heat transports for the full water column and below the mixed layer

In [33]:
Ts_JJA = xr.open_mfdataset(path + "post/THTs.JJA.0201-0300.nc").rename({"z": "Z", "lat": "YG"})
Ms_JJA_NoML = xr.open_mfdataset(path + "post/MHTsNoML.JJA.0201-0300.nc")
Ts_JJA_NoML = xr.open_mfdataset(path + "post/THTsNoMLJJA.JJA.0201-0300.nc").drop("z")
iso_JJA_NoML = xr.open_mfdataset(path + "post/HTisoNoMLjja_eddy.JJA.0201-0300.nc").drop("z")

In [34]:
transient_HT_JJA = xr.Dataset(coords={"YG": ds_1y.YG, "Z": ds_1y.Z})
transient_HT_JJA["THTdiv"] = (Ts_JJA.THTdiv * hfac * 1e-12).mean("time")
transient_HT_JJA["THTdiveddy"] = (Ts_JJA.THTdiveddy * hfac * 1e-12).mean("time")

In [35]:
mean_HT_JJA_NoML = xr.Dataset(coords={"YG": ds_1y.YG, "Z": ds_1y.Z})
mean_HT_JJA_NoML["MHT"] = (Ms_JJA_NoML.MHT * hfac * 1e-12).mean("time")

In [ ]:
transient_HT_JJA_NoML = xr.Dataset(coords={"YG": ds_1y.YG, "Z": ds_1y.Z})
transient_HT_JJA_NoML["THTdiv"] = (Ts_JJA_NoML.THTdiv * hfac * 1e-12).mean("time")
transient_HT_JJA_NoML["THTdiveddy"] = (Ts_JJA_NoML.THTdiveddy * hfac * 1e-12).mean("time")

For the isopycnal heat transports, we also need those remapped to z-levels:

In [37]:
layer_bounds = np.hstack(([0., 10., 20., 25., 30., 31.], 
                           np.arange(32., 35., 0.1),
                           np.arange(35., 36., 0.05),
                           np.arange(36., 36.9, 0.02),
                           np.arange(36.9, 37.199, 0.01),
                           np.arange(37.2, 37.6, 0.02),
                           np.arange(37.6, 37.8, 0.01),
                           np.arange(37.8, 37.899, 0.05),
                           [37.9, 38., 38.1, 38.3, 38.5, 39., 40., 45., 50., 100.]))

In [139]:
iso_JJA_NoML["layer_depths"] = xr.DataArray(-iso_JJA_NoML.levThick.mean("time").sortby("layer_center", 
                                                                     ascending=True).cumsum(dim="layer_center").values, 
                                       dims=("YG", "layer_center"))
iso_JJA_NoML["levThickmean"] = xr.DataArray(iso_JJA_NoML.levThick.mean("time").values, dims=("YG", "layer_center"))
metrics_tmp = {
    ('Z'): ['levThickmean'], # Z distances
    }
iso_JJA_NoML["layer_center"].attrs["axis"] = "Z"
iso_JJA_NoML["layer_bounds"] = layer_bounds
iso_JJA_NoML["layer_bounds"].attrs["axis"] = "Z"
iso_JJA_NoML["layer_bounds"].attrs["c_grid_axis_shift"] = -0.5
layer_depths_at_bounds = iso_JJA_NoML["layer_depths"].interp(layer_center=layer_bounds, kwargs={"fill_value": "extrapolate"}).rename({"layer_center": "layer_bounds"})
layer_depths_at_bounds["layer_bounds"].attrs["axis"] = "Z"
layer_depths_at_bounds["layer_bounds"].attrs["c_grid_axis_shift"] = -0.5
#layer_depths_at_bounds["layer_center"] = iso_JJA_NoML.layer_center

gridtmp = xgcm.Grid(iso_JJA_NoML, periodic=False, metrics=metrics_tmp)
exclude = ["levThick", "layer_depths", "levThickmean", "layer_bounds"]
for var in ["ADViso", "DIFFiso", "ADViso_eddyV", "DIFFiso_eddy_opt4"]:
    if ((var not in iso_JJA_NoML.coords) & (var not in exclude)):
        iso_JJA_NoML[var + "_z"] = gridtmp.transform(iso_JJA_NoML[var], "Z", ds_1y.Z[::-1], target_data=layer_depths_at_bounds, method="conservative")

/home/jrieck/.local/lib/python3.10/site-packages/xgcm/grid.py:982: FutureWarning: From version 0.8.0 the Axis computation methods will be removed, in favour of using the Grid computation methods instead. i.e. use `Grid.transform` instead of `Axis.transform`
  warnings.warn(
/home/jrieck/.local/lib/python3.10/site-packages/xgcm/transform.py:235: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  out = xr.apply_ufunc(
/home/jrieck/.local/lib/python3.10/site-packages/xgcm/grid.py:982: FutureWarning: From version 0.8.0 the Axis computation methods will be removed, in favour of using the Grid computation methods instead. i.e. use `Grid.transform` instead of `Axis.transform`
  warnings.warn(
/home/jrieck/.local/lib/python3.10/site-packages/xgcm/transform.py:235: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a 

In [148]:
isopycnal_HT_JJA_NoML = xr.Dataset(coords={"YG": ds_1y.YG, "Z": iso_JJA_NoML.Z, "layer_center": iso_JJA_NoML.layer_center})
isopycnal_HT_JJA_NoML["iso_total"] = (iso_JJA_NoML.ADViso + iso_JJA_NoML.DIFFiso * 1e-12).mean("time")
isopycnal_HT_JJA_NoML["iso_totaleddy"] = (iso_JJA_NoML.ADViso_eddyV + iso_JJA_NoML.DIFFiso_eddy_opt4 * 1e-12).mean("time")
isopycnal_HT_JJA_NoML["iso_total_z"] = (iso_JJA_NoML.ADViso_z + iso_JJA_NoML.DIFFiso_z * 1e-12).mean("time")
isopycnal_HT_JJA_NoML["iso_totaleddy_z"] = (iso_JJA_NoML.ADViso_eddyV_z + iso_JJA_NoML.DIFFiso_eddy_opt4_z * 1e-12).mean("time")
isopycnal_HT_JJA_NoML["levThick"] = iso_JJA_NoML.levThick.mean("time")

In [152]:
transient_HT_JJA.to_netcdf("transient_heat_transports_0201-01-01_0300-12-30_JJA_zonal_integral.nc")
mean_HT_JJA_NoML.to_netcdf("mean_heat_transports_0201-01-01_0300-12-30_JJA_NoML_zonal_integral.nc")
transient_HT_JJA_NoML.to_netcdf("transient_heat_transports_0201-01-01_0300-12-30_JJA_NoML_zonal_integral.nc")
isopycnal_HT_JJA_NoML.to_netcdf("isopycnal_heat_transports_0201-01-01_0300-12-30_JJA_NoML_zonal_integral.nc")

### Figure 8

The ocean-ice heat flux and vertical heat transport (`VHT`) across the depth of the mean winter mixed layer have already been calculated, we just need to integrate `VHT` zonally and convert everything to TW.

In [ ]:
VHT = xr.open_mfdataset(path + "post/VHT_across_base_of_winter_MLD.JJA.0201-0300.nc")
OIQ = xr.open_mfdataset(path + "post/OIQ.JJA.0201-0300.nc")

In [ ]:
OIQ_JJA = xr.Dataset(coords={"YG": ds_1y.YG})
OIQ_JJA["total"] = (OIQ.OIQtotal / 1e12).mean("time")
OIQ_JJA["prime"] = (OIQ.OIQprime / 1e12).mean("time")
OIQ_JJA["prime_CME"] = (OIQ.OIQprimeeddy / 1e12).mean("time")
OIQ_JJA["prime_CME_anti"] = (OIQ.OIQprimeeddy_anticyclones / 1e12).mean("time")
OIQ_JJA["prime_CME_cycl"] = (OIQ.OIQprimeeddy_cyclones / 1e12).mean("time")

In [ ]:
VHT_JJA = xr.Dataset(coords={"YG": ds_1y.YG})
VHT_JJA["total"] = ((VHT.VHTbar + VHT.VHTprime) * ds_1y.dyU.isel(XG=0) / 1e12).mean("time")
VHT_JJA["prime"] = (VHT.VHTprime * ds_1y.dyU.isel(XG=0) / 1e12).mean("time")
VHT_JJA["CME"] = (VHT.VHTeddy * ds_1y.dyU.isel(XG=0) / 1e12).mean("time")
VHT_JJA["CME_anti"] = (VHT.VHTeddy_anticyclones * ds_1y.dyU.isel(XG=0) / 1e12).mean("time")
VHT_JJA["CME_cycl"] = (VHT.VHTeddy_cyclones * ds_1y.dyU.isel(XG=0) / 1e12).mean("time")
VHT_JJA["DIFF"] = (VHT.VHTDIFFbar / 1e12).mean("time")

In [ ]:
OIQ_JJA.to_netcdf("ocean_ice_heat_transports_0201-01-01_0300-12-30_JJA_zonal_integral.nc")
VHT_JJA.to_netcdf("vertical_heat_transports_0201-01-01_0300-12-30_JJA_zonal_integral.nc")

### Figure 9

Nothing to prepare.

### Figure S1

Compute Rossby readius for $L_{d}/\Delta x$, with $L_{d}$ being the deformation scale and $\Delta x$ the horizontal grid spacing.

In [45]:
g = 9.81
f0 = -1.405e-4
beta = 1.145e-11
f = f0 + beta * ds_1y.YC

In [48]:
N = np.zeros((len(ds_1y.time), 53, len(ds_1y.YC), len(ds_1y.XC)))
dz = np.zeros((53))
for k in np.arange(1, 54):
    dk = ds_1y.isel(Z=slice(k-1, k+2))
    rho = xr.apply_ufunc(jmd.dens, dk.SALT, dk.THETA, -ds_1y.isel(Z=k).Z.values,
                         dask='parallelized',
                         output_dtypes=[dk.THETA.dtype], keep_attrs=True).data
    dz[k-1] = dk.Z.data[0] - dk.Z.data[2]
    drhodz = (rho[:, 0, :, :] - rho[:, 2, :, :]) / dz[k-1]
    N2 = (-(g / rho[:, 1, :, :]) * drhodz)
    N2[N2 < 0.] = 0.
    N[:, k-1, :, :] = np.sqrt(N2)
    
Rossby = (1 / (np.pi * np.abs(f.values)))[None, None, :, None] * np.nansum(N[:, ::-1, :, :] * dz[None, ::-1, None, None], axis=1)

In [57]:
meanRossby = xr.DataArray(np.nanmean(Rossby, axis=1).squeeze(), 
                          dims={"YC": ds_1y.YC, "XC": ds_1y.XC}, 
                          coords={"YC": ds_1y.YC, "XC": ds_1y.XC}, name="Ro")

In [58]:
meanRossby.to_netcdf("meanRossbyRadius.nc")

### Figure S2

Spectra of EKE at different latitudes

In [39]:
vbar = ds_5d.sel(time=slice("0291-01-01", "0300-12-30")).isel(Z=15).VVEL.groupby('time.month').mean("time")
vprime = ds_5d.sel(time=slice("0291-01-01", "0300-12-30")).isel(Z=15).VVEL.groupby('time.month') - vbar

In [40]:
power = xrft.power_spectrum(vprime.drop(['drS', 'dxG', 'dyC', 'hFacS', 'maskS', 'rAs']), dim=['XC'])
powereke = .5*(power).mean('time').isel(freq_XC=slice(int(len(ds_5d.XC)/2.)+1, None)).rename("power")
powereke["wavenumber"] = (powereke.freq_XC * 1e3  * np.pi)

In [41]:
powereke.to_netcdf("zonal_eke_spectra_0291-0300.nc")

### Figure S3

Examples of the eddy masks $M^{CME}$ with different detection methods or parameters.

In [9]:
y = "0296"
t_snap = y + '-03-03'

In [10]:
emOW02 = xr.open_mfdataset(path + "post/eddymask_binary_" + y + "_k15.0.2.nc")
emOW03 = xr.open_mfdataset(path + "post/eddymask_binary_" + y + "_k15.0.3.nc")
emOW04 = xr.open_mfdataset(path + "post/eddymask_binary_" + y + "_k15.0.4.nc")
emOW05 = xr.open_mfdataset(path + "post/eddymask_binary_" + y + "_k15.0.5.nc")
emSSH = xr.open_mfdataset(path + "post/eddymask_binary_" + y + ".SSH.nc")
emUV = xr.open_mfdataset(path + "post/eddymask_binary_" + y + "_k15.UV.nc")
em3D = xr.open_mfdataset(path + "post/eddymask_binary_" + y + ".3D.nc")

In [11]:
em_baro = xr.open_mfdataset(path + "post/eddymask_binary_????_k15.0.3.nc")
em_3D = xr.open_mfdataset(path + "post/eddymask_binary_????.3D.nc")
em_chelton = emSSH = xr.open_mfdataset(path + "post/eddymask_binary_????.SSH.nc")
em_nencioli = emUV = xr.open_mfdataset(path + "post/eddymask_binary_????_k15.UV.nc")

Compute the volume of the domain occupied by CME based on the different eddy masks $M^{CME}$

In [12]:
shelf = ds_5d.maskShelf.rename({"XC": "lon", "YG": "lat", "Z": "z"})
shelf["lon"] = emOW03.lon

In [13]:
weighted_baro = em_baro.eddymask_binary.mean("z").expand_dims(dim={"z": em3D.z}).where(shelf==1) * shelf.drF * shelf.dxG * shelf.dyC
volume_baro = weighted_baro.sum(("z", "lon", "lat")).mean("time")
weighted_3D = em_3D.eddymask_binary.where(shelf==1) * shelf.drF * shelf.dxG * shelf.dyC
volume_3D = weighted_3D.sum(("z", "lon", "lat")).mean("time")
weighted_chelton = em_chelton.eddymask_binary.expand_dims(dim={"z": em3D.z}).where(shelf==1) * shelf.drF * shelf.dxG * shelf.dyC
volume_chelton = weighted_chelton.sum(("z", "lon", "lat")).mean("time")
weighted_nencioli = em_nencioli.eddymask_binary.mean("z").expand_dims(dim={"z": em3D.z}).where(shelf==1) * shelf.drF * shelf.dxG * shelf.dyC
volume_nencioli = weighted_nencioli.sum(("z", "lon", "lat")).mean("time")

Volume of the full domain

In [14]:
weighted_all = em_3D.eddymask_binary.where(shelf==0, other=1) * shelf.drF * shelf.dxG * shelf.dyC
volume_all = weighted_all.sum(("z", "lon", "lat")).mean("time")

In [15]:
occupied_baro = (volume_baro / volume_all).values
occupied_3D = (volume_3D / volume_all).values
occupied_chelton = (volume_chelton / volume_all).values
occupied_nencioli = (volume_nencioli / volume_all).values

In [20]:
print("CME from the barotropic Okubo-Weiss M^CME occupy", str(np.around(occupied_baro * 100, decimals=1)), "% of the domain")
print("CME from the 3D Okubo-Weiss M^CME occupy", str(np.around(occupied_3D * 100, decimals=1)), "% of the domain")
print("CME from the Chelton, SSH-based M^CME occupy", str(np.around(occupied_chelton * 100, decimals=1)), "% of the domain")
print("CME from the Nencioli, velcoity-based M^CME occupy", str(np.around(occupied_nencioli * 100, decimals=1)), "% of the domain")

CME from the barotropic Okubo-Weiss M^CME occupy 19.5 % of the domain
CME from the 3D Okubo-Weiss M^CME occupy 22.0 % of the domain
CME from the Chelton, SSH-based M^CME occupy 15.3 % of the domain
CME from the Nencioli, velcoity-based M^CME occupy 10.2 % of the domain


In [28]:
emOW02_snap = emOW02.sel(time=t_snap, method="nearest").rename({"eddymask_binary": "emOW02"}).mean(("time", "z"))
emOW03_snap = emOW03.sel(time=t_snap, method="nearest").rename({"eddymask_binary": "emOW03"}).mean(("time", "z"))
emOW04_snap = emOW04.sel(time=t_snap, method="nearest").rename({"eddymask_binary": "emOW04"}).mean(("time", "z"))
emOW05_snap = emOW05.sel(time=t_snap, method="nearest").rename({"eddymask_binary": "emOW05"}).mean(("time", "z"))
emSSH_snap = emSSH.sel(time=t_snap, method="nearest").rename({"eddymask_binary": "emSSH"}).mean(("time"))
emUV_snap = emUV.sel(time=t_snap, method="nearest").rename({"eddymask_binary": "emUV"}).mean(("time", "z"))
em3D_snap = em3D.sel(time=t_snap, method="nearest").rename({"eddymask_binary": "em3D"}).mean(("time"))

In [29]:
eddymasks = xr.merge([emOW02_snap, emOW03_snap, emOW04_snap, emOW05_snap, emSSH_snap, emUV_snap, em3D_snap])

In [30]:
eddymasks.to_netcdf("eddymasks_example.nc")

We want to plot some fields of Okubo-Weiss parameter, SSH and velocity, so we use fields that were interpolated to the F-points like the masks.

In [101]:
OW_int = xr.open_mfdataset(path + "post/interp_data.extend800." + y + "*0.nc").isel(lon=slice(0, 240))
SSH_int = xr.open_mfdataset(path + "post/interp_SSH.extend800." + y + "*0.nc").isel(lon=slice(0, 240))
UV_int = xr.open_mfdataset(path + "post/interp_UV.extend800." + y + "*0.nc").isel(lon=slice(0, 240))

In [107]:
OW_snap = OW_int.OW.sel(time=t_snap, method="nearest").isel(z=15).mean("time")
SSH_snap = SSH_int.ETAN.sel(time=t_snap, method="nearest").mean("time") - SSH_int.ETAN.mean("time")
UV_snap = UV_int.sel(time=t_snap, method="nearest").isel(z=15).mean("time").drop(["Depth", "dxV", "dyU", "maskZ", "maskC", "maskW", "maskS"])

In [109]:
OW_snap.to_netcdf("OkuboWeiss_example.nc")

In [110]:
SSH_snap.to_netcdf("SSH_example.nc")

In [111]:
UV_snap.to_netcdf("UV_example.nc")

### Figure S4

Sensitivity of the zonally and vertically integrated $THT$ ot different detection methods or parameters. 

In [62]:
Ts02 = xr.open_mfdataset(path + 'post/THTs.0.2.02910101_03001230.nc')
Ts03 = xr.open_mfdataset(path + 'post/THTs.0.3.02910101_03001230.nc')
Ts04 = xr.open_mfdataset(path + 'post/THTs.0.4.02910101_03001230.nc')
Ts05 = xr.open_mfdataset(path + 'post/THTs.0.5.02910101_03001230.nc')
TsSSH = xr.open_mfdataset(path + 'post/THTs.SSH.02910101_03001230.nc')
TsUV = xr.open_mfdataset(path + 'post/THTs.UV.02910101_03001230.nc')
Ts3D = xr.open_mfdataset(path + 'post/THTs.3D.0.3.02910101_03001230.nc')
HTtrap = HTtrap.sel(time=slice("0291-01-01", "0300-12-30"))

In [48]:
for ds in [Ts02, Ts03, Ts04, Ts05, TsUV]:
    ds = ds.drop("z").rename({"Z": "z", "YG": "lat"})
    
TsSSH = TsSSH.rename({"Z": "z", "YG": "lat"})
Ts3D = Ts3D.rename({"Z": "z", "YG": "lat"})

In [81]:
sensitivity_HT = xr.Dataset(coords={"YG": ds_1y.YG})
sensitivity_HT["THTdiv"] = ((Ts03.THTdiv * ds_1y.drF.values[:, None] * hfac).sum("Z") * 1e-12)
sensitivity_HT["THTobs"] = ((HTtrap["THTobs"] * ds_1y.drF.values[None, :] * hfac.T).sum("z") * 1e-12)
sensitivity_HT["THTdiveddyOW02"] = ((Ts02.THTdiveddy * ds_1y.drF.values[:, None] * hfac).sum("Z") * 1e-12)
sensitivity_HT["THTdiveddyOW03"] = ((Ts03.THTdiveddy * ds_1y.drF.values[:, None] * hfac).sum("Z") * 1e-12)
sensitivity_HT["THTdiveddyOW04"] = ((Ts04.THTdiveddy * ds_1y.drF.values[:, None] * hfac).sum("Z") * 1e-12)
sensitivity_HT["THTdiveddyOW05"] = ((Ts05.THTdiveddy * ds_1y.drF.values[:, None] * hfac).sum("Z") * 1e-12)
sensitivity_HT["THTdiveddySSH"] = ((TsSSH.THTdiveddy * ds_1y.drF.values[:, None] * hfac).sum("Z") * 1e-12)
sensitivity_HT["THTdiveddyUV"] = ((TsUV.THTdiveddy * ds_1y.drF.values[:, None] * hfac).sum("Z") * 1e-12)
sensitivity_HT["THTdiveddy3D"] = ((Ts3D.THTdiveddy * ds_1y.drF.values[:, None] * hfac).sum("Z") * 1e-12)

In [83]:
sensitivity_HT.to_netcdf("sensitivity_detection.nc")

### Figure S5

Same as for Figure 4, 5 and 6 but for the trapping component of HT split into contributions by anticyclones and cyclones.

In [ ]:
HTtrap_anticyclones = xr.open_mfdataset(path + "post/THTobs_anticyclones.0201-0300.nc", 
                                        concat_dim="time", combine="nested",  data_vars='minimal', 
                                        coords='minimal', compat='override').rename({"lat": "YG", "z": "Z"})
HTtrap_cyclones = xr.open_mfdataset(path + "post/THTobs_cyclones.0201-0300.nc", 
                                    concat_dim="time", combine="nested",  data_vars='minimal', 
                                    coords='minimal', compat='override').rename({"lat": "YG", "z": "Z"})

In [ ]:
obs_like_HT = xr.Dataset(coords={"YG": ds_1y.YG})
obs_like_HT["THTtrap"] = ((HTtrap["THTobs"] * dz * hfac.T).sum("Z") * 1e-12).mean("time")
obs_like_HT["THTtrapstd"] = ((HTtrap["THTobs"] * dz * hfac.T).sum("Z") * 1e-12).std("time")
obs_like_HT["THTtrap_anticyclones"] = ((HTtrap_anticyclones["THTobs"] * dz * hfac.T).sum("Z") * 1e-12).mean("time")
obs_like_HT["THTtrap_anticyclonesstd"] = ((HTtrap_anticyclones["THTobs"] * dz * hfac.T).sum("Z") * 1e-12).std("time")
obs_like_HT["THTtrap_cyclones"] = ((HTtrap_cyclones["THTobs"] * dz * hfac.T).sum("Z") * 1e-12).mean("time")
obs_like_HT["THTtrap_cyclonesstd"] = ((HTtrap_cyclones["THTobs"] * dz * hfac.T).sum("Z") * 1e-12).std("time")

In [ ]:
obs_like_HT.to_netcdf("obs_like_heat_transports_0201-01-01_0300-12-30_zonal_vertical_integral.nc")

### Figure S6

Same as for Figure 4, 5 and 6 but for the contribution to isopycnal heat transport by CME split up into anticyclones and cyclones.

In [40]:
iso = xr.open_mfdataset(path + "run/post/HTisoNoMLjja_eddy.0201-0300.nc").drop("z")

In [ ]:
isopycnal_eddy_HT = xr.Dataset(coords={"YG": ds_1y.YG})
isopycnal_eddy_HT["iso_total"] = ((iso.ADViso + iso.DIFFiso).sum("layer_center") * 1e-12).mean("time")
isopycnal_eddy_HT["iso_totalstd"] = ((iso.ADViso + iso.DIFFiso).sum("layer_center") * 1e-12).std("time")
isopycnal_eddy_HT["iso_totaleddy"] = ((iso.ADViso_eddyV 
                                       + iso.DIFFiso_eddy_opt4).sum("layer_center") * 1e-12).mean("time")
isopycnal_eddy_HT["iso_totaleddystd"] = ((iso.ADViso_eddyV 
                                          + iso.DIFFiso_eddy_opt4).sum("layer_center") * 1e-12).std("time")
isopycnal_eddy_HT["iso_totaleddy_anticyclones"] = ((iso.ADViso_eddyV_anticyclones 
                                                    + iso.DIFFiso_eddy_anticyclones_opt4).sum("layer_center") * 1e-12).mean("time")
isopycnal_eddy_HT["iso_totaleddy_anticyclonesstd"] = ((iso.ADViso_eddyV_anticyclones 
                                                       + iso.DIFFiso_eddy_anticyclones_opt4).sum("layer_center") * 1e-12).std("time")
isopycnal_eddy_HT["iso_totaleddy_cyclones"] = ((iso.ADViso_eddyV_cyclones 
                                                + iso.DIFFiso_eddy_cyclones_opt4).sum("layer_center") * 1e-12).mean("time")
isopycnal_eddy_HT["iso_totaleddy_cyclonesstd"] = ((iso.ADViso_eddyV_cyclones 
                                                   + iso.DIFFiso_eddy_cyclones_opt4).sum("layer_center") * 1e-12).std("time")

In [ ]:
isopycnal_eddy_HT.to_netcdf("isopycnal_eddy_heat_transports_0201-01-01_0300-12-30_zonal_vertical_integral.nc")

### Figure S7

Same as for Figure 4, 5 and 6 but for the contribution to isopycnal heat transport by "advectve" and "diffusice" processes (as defined by Lee et al., 2007).

In [41]:
iso["layer_depths"] = xr.DataArray(-iso.levThick.mean("time").sortby("layer_center", 
                                                                     ascending=True).cumsum(dim="layer_center").values, 
                                       dims=("YG", "layer_center"))
iso["levThickmean"] = xr.DataArray(iso.levThick.mean("time").values, dims=("YG", "layer_center"))
metrics_tmp = {
    ('Z'): ['levThickmean'], # Z distances
    }
iso["layer_center"].attrs["axis"] = "Z"
iso["layer_bounds"] = layer_bounds
iso["layer_bounds"].attrs["axis"] = "Z"
iso["layer_bounds"].attrs["c_grid_axis_shift"] = -0.5
layer_depths_at_bounds = iso["layer_depths"].interp(layer_center=layer_bounds, kwargs={"fill_value": "extrapolate"}).rename({"layer_center": "layer_bounds"})
layer_depths_at_bounds["layer_bounds"].attrs["axis"] = "Z"
layer_depths_at_bounds["layer_bounds"].attrs["c_grid_axis_shift"] = -0.5

gridtmp = xgcm.Grid(iso, periodic=False, metrics=metrics_tmp)
exclude = ["levThick", "layer_depths", "levThickmean", "layer_bounds"]
for var in ["ADViso", "DIFFiso", "ADViso_eddyV", "DIFFiso_eddy_opt4"]:
    if ((var not in iso.coords) & (var not in exclude)):
        iso[var + "_z"] = gridtmp.transform(iso[var], "Z", ds_1y.Z[::-1], target_data=layer_depths_at_bounds, method="conservative")

/home/jrieck/.venv/py_eddytools/lib/python3.8/site-packages/xgcm/transform.py:235: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  out = xr.apply_ufunc(
/home/jrieck/.venv/py_eddytools/lib/python3.8/site-packages/xgcm/transform.py:235: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  out = xr.apply_ufunc(
/home/jrieck/.venv/py_eddytools/lib/python3.8/site-packages/xgcm/transform.py:235: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  out = xr.apply_ufunc(
/home/jrieck/.venv/py_eddytools/lib/python3.8/site-packages/xgcm/transform.py:235: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.


In [42]:
iso_eddy_HT_detail = xr.Dataset(coords={"YG": ds_1y.YG})
iso_eddy_HT_detail["iso_ADV"] = (iso.ADViso_eddyV * 1e-12).mean("time")
iso_eddy_HT_detail["iso_ADVstd"] = (iso.ADViso_eddyV * 1e-12).std("time")
iso_eddy_HT_detail["iso_DIFF"] = (iso.DIFFiso_eddy_opt4 * 1e-12).mean("time")
iso_eddy_HT_detail["iso_DIFFstd"] = (iso.DIFFiso_eddy_opt4 * 1e-12).std("time")
iso_eddy_HT_detail["iso_ADV_z"] = (iso.ADViso_eddyV_z * 1e-12).mean("time")
iso_eddy_HT_detail["iso_ADVstd_z"] = (iso.ADViso_eddyV_z * 1e-12).std("time")
iso_eddy_HT_detail["iso_DIFF_z"] = (iso.DIFFiso_eddy_opt4_z * 1e-12).mean("time")
iso_eddy_HT_detail["iso_DIFFstd_z"] = (iso.DIFFiso_eddy_opt4_z * 1e-12).std("time")

In [44]:
iso_eddy_HT_detail.to_netcdf("isopycnal_eddy_heat_transports_details_0201-01-01_0300-12-30_zonal_integral_on_z.nc")

In [ ]:
ds_5d.MXLDEPTH.max(("time", "XC")).to_netcdf("MXLDEPTH_0201-01-01_0300-12-30_max_zonal_max.nc")

### Figure S8

Count the number of anticyclones and cyclones at each latitude.

In [ ]:
eddies_time_range = xr.cftime_range(start='0201-01-03 00', end='0300-12-30 00', 
                                    calendar='360_day', freq=str(5) + 'D')
etr = [str(eddies_time_range[t]) for t in range(0, len(eddies_time_range))]
eddies = {}
for i in np.arange(0, len(etr)):
    datestring = etr[i][0:10]
    with open(path + 'tracking/eddies/eddies_'
              + datestring + '.pickle', 'rb') as f:
        eddies[i] = pickle.load(f)
    f.close()

In [ ]:
number = 0
time = []
typ = []
lon = []
lat = []
eddy_i = []
eddy_j = []
area = []
scale = []
amp = []
for j in np.arange(0, len(eddies)):
    number += len(eddies[j])
    time.append(np.stack([eddies[j][k]["time"] for k in np.arange(0, len(eddies[j]))]))
    typ.append(np.stack([eddies[j][k]["type"] for k in np.arange(0, len(eddies[j]))]))
    lon.append(np.stack([eddies[j][k]["lon"][0] for k in np.arange(0, len(eddies[j]))]))
    lat.append(np.stack([eddies[j][k]["lat"][0] for k in np.arange(0, len(eddies[j]))]))
    eddy_i.append(np.stack([int(np.median(eddies[j][k]["eddy_i"])) for k in np.arange(0, len(eddies[j]))]))
    eddy_j.append(np.stack([int(np.median(eddies[j][k]["eddy_j"])) for k in np.arange(0, len(eddies[j]))]))
    area.append(np.stack([eddies[j][k]["area"][0] for k in np.arange(0, len(eddies[j]))]))
    scale.append(np.stack([eddies[j][k]["scale"][0][0] for k in np.arange(0, len(eddies[j]))]))
    amp.append(np.stack([eddies[j][k]["amp"][0] for k in np.arange(0, len(eddies[j]))]))

all_eddies = xr.Dataset({"number": (["number"], np.arange(0, number)),
                         "time": (["number"], np.hstack(time)),
                         "type": (["number"], np.hstack(typ)),
                         "lon": (["number"], np.hstack(lon)),
                         "lat": (["number"], np.hstack(lat)),
                         "eddy_i": (["number"], np.hstack(eddy_i)),
                         "eddy_j": (["number"], np.hstack(eddy_j)),
                         "scale": (["number"], np.hstack(scale)),
                         "amp": (["number"], np.hstack(amp))
                        })

all_eddies["SIarea"] = ds_5d.SIarea.isel(XC=all_eddies.eddy_i, 
                                         YC=all_eddies.eddy_j).sel(time=all_eddies.time, 
                                                              method="nearest").drop(["XC", "YC", "time"])

all_eddies.to_netcdf("all_eddies_02010101_03001230.nc")

In [ ]:
cycl_eddy = all_eddies.where(all_eddies.type == 'cyclonic')
anti_eddy = all_eddies.where(all_eddies.type == 'anticyclonic')

In [ ]:
lat_bins = np.linspace(0, 3.2e6, 320)
lat_labels = (lat_bins[0:-1] + lat_bins[1::]) / 2
lats = xr.Dataset(coords={"lat": lat_labels})
                  
lats["number_cyc"] = cycl_eddy.dropna("number").groupby_bins("lat", bins=lat_bins, 
                                                        labels=lat_labels).count().lat.rename({"lat_bins": "lat"})
lats["number_anti"] = anti_eddy.dropna("number").groupby_bins("lat", bins=lat_bins, 
                                                         labels=lat_labels).count().lat.rename({"lat_bins": "lat"})

In [ ]:
lats.to_netcdf("eddy_lat_distribution.nc")